In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# basic_chat_gradio

> A basic gradio chat bot using OpenAI API's

In [ ]:
#| default_exp basic_chat_gradio

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| exporti
import jarvis.secrets

In [ ]:
#| exporti
import openai
import numpy as np
import os
import json
import gradio as gr
import datetime

Setting the default environment API key for the OpenAI API python client.

In [ ]:
#| exporti
openai.api_key = os.environ["OPENAI_API_KEY"]

List of all OpenAI model listing the root model and release date, sorted by release date with the most recent at the top.

In [ ]:
models = openai.Model.list()
for _model in sorted(models.data, key=lambda o: o.get('created'), reverse=True):
    print(f"{_model.get('id')} - ROOT: {_model.get('root')} - CREATED: {datetime.datetime.fromtimestamp(_model.get('created'))}")
#models.data

gpt-4 - ROOT: gpt-4 - CREATED: 2023-03-12 07:03:22
gpt-4-0314 - ROOT: gpt-4-0314 - CREATED: 2023-03-12 07:03:21
gpt-3.5-turbo-0301 - ROOT: gpt-3.5-turbo-0301 - CREATED: 2023-03-01 05:52:43
gpt-3.5-turbo - ROOT: gpt-3.5-turbo - CREATED: 2023-02-28 18:56:42
whisper-1 - ROOT: whisper-1 - CREATED: 2023-02-27 21:13:04
text-embedding-ada-002 - ROOT: text-embedding-ada-002 - CREATED: 2022-12-16 19:01:39
text-davinci-003 - ROOT: text-davinci-003 - CREATED: 2022-11-28 01:40:35
ada-code-search-text - ROOT: ada-code-search-text - CREATED: 2022-04-28 19:01:50
babbage-search-document - ROOT: babbage-search-document - CREATED: 2022-04-28 19:01:50
curie-similarity - ROOT: curie-similarity - CREATED: 2022-04-28 19:01:50
babbage-code-search-code - ROOT: babbage-code-search-code - CREATED: 2022-04-28 19:01:49
babbage-code-search-text - ROOT: babbage-code-search-text - CREATED: 2022-04-28 19:01:49
davinci-search-document - ROOT: davinci-search-document - CREATED: 2022-04-28 19:01:49
curie-search-query - 

## Completion Messages

An example set of chat messages formatted for chat completion. Example from https://platform.openai.com/docs/guides/chat/introduction:
```python
openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
```
With corresponding example response (non-streaming). Example from https://platform.openai.com/docs/api-reference/chat/create
```python
{
  "id": "chatcmpl-123",
  "object": "chat.completion",
  "created": 1677652288,
  "choices": [{
    "index": 0,
    "message": {
      "role": "assistant",
      "content": "\n\nHello there, how may I assist you today?",
    },
    "finish_reason": "stop"
  }],
  "usage": {
    "prompt_tokens": 9,
    "completion_tokens": 12,
    "total_tokens": 21
  }
}
```
And finally an example streaming implementation https://huggingface.co/spaces/ysharma/ChatGPTwithAPI/blob/main/app.py

Below is a simple example of using Chat completions

In [ ]:
completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": "Hello world!"}])
print(completion.choices[0].message.content)

Hello there, how can I assist you today?


And here is a streaming example

In [ ]:
#streaming completion
response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[
        {'role': 'user', 'content': "write a short, single verse, poem about math"}
    ],
    temperature=0,
    stream=True  # this time, we set stream=True
)

for idx, chunk in enumerate(response):
    if idx == 1: print(chunk)
    print(chunk.get('choices',[{}])[0].get('delta',{}).get('content',''),end='')

{
  "choices": [
    {
      "delta": {
        "content": "Numbers"
      },
      "finish_reason": null,
      "index": 0
    }
  ],
  "created": 1681257722,
  "id": "chatcmpl-74IDyxGMJpIhakCDEALergWPVDBg9",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion.chunk"
}
Numbers dance in perfect time,
Equations flow like sweetest rhyme,
Mathematics, pure and true,
A world of wonder, ever anew.

## Gradio Interface

`parse_codeblock` is a temporary fix for gradio displaying code correctly inside of triple backticks.

In [ ]:
#| exports
def parse_codeblock(text):
    """Fixes how code blocks are displayed in the gradio chat window. 
    Ref: https://github.com/gradio-app/gradio/issues/3531"""
    lines = text.split("\n")
    for i, line in enumerate(lines):
        if "```" in line:
            if line != "```":
                lines[i] = f'<pre><code class="{lines[i][3:]}">'
            else:
                lines[i] = '</code></pre>'
        else:
            if i > 0:
                lines[i] = "<br/>" + line.replace("<", "&lt;").replace(">", "&gt;")
    return "".join(lines)

TODO: 
- Add tiktoken token counter for streaming api. https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
- Add running total calculator to track tokens and $.

In [ ]:
#|exports
def gradio_chat_history_to_openai_chat_completions_format(
    history:list[list[dict]] #Gradio chat history.
)->list[dict]:
    """Converts gradio chat history to OpenAI messages completions format."""
    chat_formatted_list = []
    for message_pair in history:
        chat_formatted_list.append({'role':'user', 'content': message_pair[0]})
        if message_pair[1] is not None:
            chat_formatted_list.append({'role':'assistant', 'content': message_pair[1]})
    return chat_formatted_list

In [ ]:
#|exports
def process_user_message(
    user_message:str, #Most recent message from user.
    history:list[list[str]] #Message history from gradio.Chatbot instance.
)->tuple[str,list[list[str]]]:
    """Add user message to 'history' and clear message textbox."""
    print('process_user_message',user_message,history)
    # if history is None: history = []
    return "", history + [[user_message, None]]

In [ ]:
#|exports
def process_bot_message(
    history:list[list[str]], #Message history from gradio.Chatbot instance.
    system_msg:str, #Most recent message from user.
    selected_model:str, #Selected OpenAI model for chat completion.
    temperature:float=0., #How deterministic the model results are.  See OpenAI docs.
    stream:bool=True, #Whether or not to stream the results or return all at once when the model has finished its response.
)->list[list[str]]:
    """Send chat history with most recent user message to the chat completions API."""
    stream_results = ''
    history[-1][1] = ''
    # try:
    response = openai.ChatCompletion.create(
        model=selected_model,
        messages=[{'role':'system', 'content': system_msg}] + gradio_chat_history_to_openai_chat_completions_format(history),
        temperature=temperature,
        stream=stream  # this time, we set stream=True
    )
    if stream:
        for chunk in response:
            model_results.append(model_results)
            # print(chunk)
            # print(chunk.get('choices',[{}])[0].get('delta',{}).get('content',''),end='')
            stream_results += chunk.get('choices',[{}])[0].get('delta',{}).get('content','')
            history[-1][1] = parse_codeblock(stream_results)
            yield history
    else:
        # print(response)
        # print(response.get('choices',[{}])[0].get('message',{}).get('content',''))
        # print(parse_codeblock(response.get('choices',[{}])[0].get('message',{}).get('content','')))
        history[-1][1] = parse_codeblock(response.get('choices',[{}])[0].get('message',{}).get('content',''))
        # print(history)
        yield history
    # except:
    #     pass
    #     # print(history, system_msg, [{'role':'system', 'content': system_msg}] + history_to_chatgpt_format(history))

    # print(response)
    # print(stream_results)
    # print(parse_codeblock(stream_results))
    # print(history[-1][1])

In [ ]:
model_results = []
with gr.Blocks(title='Jarvis - My ChatGPT') as demo:
    with gr.Tab(label='Chat'):
        chatbot = gr.Chatbot()
        # chatbot.change(fn=lambda: None, scroll_to_output=True) #This also doesn't work. Autoscroll when chatbot in tab is known issue.
        with gr.Row():
            with gr.Column(scale=10):
                msg = gr.Textbox()
            with gr.Column(scale=1):
                clear = gr.Button("Clear")
                submit = gr.Button("Send")
    with gr.Tab(label='Settings'):
        system_msg = gr.Textbox(value='You are a helpful assistant.', info='This is the system message for OpenAI Chat APIs.')
        model_selection = gr.Dropdown(choices=['gpt-3.5-turbo','gpt-4'],value='gpt-3.5-turbo',label='Select model: ')
        temperature = gr.Slider(maximum=2., value=0., step=.1, label='Temperature', info='What sampling temperature to use, between 0 and 2. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.',)
        stream_results = gr.Checkbox(value=True, label='Stream Results', )

    submit0_kwargs = dict(fn=process_user_message, inputs=[msg, chatbot], outputs=[msg, chatbot])
    submit1_kwargs = dict(fn=process_bot_message, inputs=[chatbot, system_msg, model_selection, temperature, stream_results], outputs=chatbot)
    msg.submit(**submit0_kwargs).then(**submit1_kwargs)
    clear.click(fn=lambda: (None, None), inputs=None, outputs=[chatbot,msg], queue=False)
    submit.click(**submit0_kwargs).then(**submit1_kwargs)

In [ ]:
#| eval: false
#| output: false
demo.queue(concurrency_count=2)
demo.launch(server_name='0.0.0.0')#,server_port=7860)

In [ ]:
#| eval: false
demo.close()

Closing server running on port: 7862


In [ ]:
#| eval: false
gr.close_all()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()